# init
used to query DB for spectra and labels

In [11]:
import os
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from datetime import datetime
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
from pptoolbox.connectors import BaseEFSConnector, PFDBConnector

In [3]:
load_dotenv(find_dotenv())

PF_SQL_PASSWORD = os.environ.get("PLATFORM_SQL_PASSWORD", None)  # PP_SQL_PASSWORD
PF_KEY_PATH = os.environ.get("DS_SERVER_KEYPATH", None)  # PP_SERVER_KEYPATH
PF_EFS_URL = os.environ.get("PF_EFS_URL", None)  # PP_EFS_URL

print(PF_SQL_PASSWORD,PF_KEY_PATH,PF_EFS_URL)

F398FYMLKN7XVVXJ C:/Users/RyanSoh/.ssh/datascience-research-instance-2.pem http://54.254.245.207:5014


In [7]:
raw_data_path = Path ("../data/raw")
output_folder = raw_data_path / "v1"
output_folder.mkdir(parents=True, exist_ok=True)

# by product_type

In [8]:
company_id = 1385
product_type = 6971
today_date = datetime.now().strftime("%y%m%d")

print (today_date)

251110


## label

In [ ]:
BASE_QUERY = f"""
WITH combined_values AS (
    SELECT lnpv.lot_id, np.property_id, lnpv.value
    FROM lot_numerical_property_value lnpv
    JOIN numerical_property np ON lnpv.numerical_property_id = np.id
    
    UNION ALL
    
    SELECT lopv.lot_id, op.property_id, lopv.value
    FROM lot_option_property_value lopv
    JOIN option_property op ON lopv.option_property_id = op.id
)

SELECT 
    cv.lot_id, 
    l.name AS lot_name, 
    p.name AS property_name,
    cv.value AS property_value,
    l.company_id,
    l.product_type_id,
    pt.name AS product_name
FROM combined_values cv
INNER JOIN lot l ON l.id = cv.lot_id
INNER JOIN property p ON p.id = cv.property_id
INNER JOIN product_type pt ON pt.id = l.product_type_id
WHERE l.company_id = {company_id}
  AND l.product_type_id = {product_type}
ORDER BY cv.lot_id
"""

In [ ]:
# BASE_QUERY = f"""
# SELECT lnpv.lot_id, lot.name as lot_name, 
# 	   property.name as property_name,
#        lnpv.value as property_value,
#        lot.company_id,
#        lot.product_type_id,
#        product_type.name as product_name
# FROM lot_numerical_property_value lnpv
# INNER JOIN lot on lot.id = lnpv.lot_id
# INNER JOIN numerical_property on numerical_property.id = lnpv.numerical_property_id
# INNER JOIN property on numerical_property.property_id = property.id
# INNER JOIN product_type on product_type.id = lot.product_type_id
# WHERE lot.company_id = {company_id}
# AND lot.product_type_id = {product_type}

# UNION ALL

# SELECT lopv.lot_id, lot.name as lot_name, 
# 	   property.name as property_name,
#        lopv.value as property_value,
#        lot.company_id,
#        lot.product_type_id,
#        product_type.name as product_name
# FROM lot_option_property_value lopv
# INNER JOIN lot on lot.id = lopv.lot_id
# INNER JOIN option_property on option_property.id = lopv.option_property_id
# INNER JOIN property on option_property.property_id = property.id
# INNER JOIN product_type on product_type.id = lot.product_type_id
# WHERE lot.company_id = {company_id}
# AND lot.product_type_id = {product_type}

# ORDER BY lot_id
# """

In [ ]:
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY).set_index("lot_id")
print("successful query")

label_filename = output_folder / f"label_{today_date}.csv"
info_df.to_csv(label_filename)

## spectra

In [9]:
BASE_QUERY = f"""
SELECT
	sp.id AS specimen_id,
	l.id AS lot_id,
	l.name AS lot_name,
	sp.date_scanned,
	sp.analyzer_id AS analyzer_id,
    l.company_id as company_id,
	p.id AS product_id,
    p.name AS product_name
FROM
	specimen sp
	INNER JOIN lot l ON l.id = sp.lot_id
	INNER JOIN product_type p on l.product_type_id = p.id
WHERE l.company_id = {company_id}
AND p.id = {product_type}
ORDER BY l.id;
"""

In [ ]:
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY)
print("successful query")
efs_conn = BaseEFSConnector(url=PF_EFS_URL)
print(len(info_df))

spectra_df = efs_conn.fetch_spectra(info_df.specimen_id.values)
column_diff = set(info_df.columns) & set(spectra_df.columns)
joined_df = spectra_df.merge(
    info_df.drop(column_diff, axis=1), right_on="specimen_id", left_index=True
)
joined_df.set_index("lot_id", inplace=True)

print("Successfully queried from DB")
display(joined_df.head(n=10))
print(joined_df.shape)

spectra_filename = output_folder / f"spectra_{today_date}.csv"
joined_df.to_csv(spectra_filename)

successful query
20


Fetching from EFS: 100%|██████████| 1/1 [00:08<00:00,  8.06s/it]

Successfully queried from DB


,active,analyzer_id,calc_data_mini,calc_data_neo,company_id,dark_ref_data_mini,dark_ref_data_neo,dark_ref_scan_time,date_scanned,detector_temp,frame_data,healthy,id,integration_time,integration_time_neo,name,scope_data_mini,scope_data_neo,spectrometer_temp_mini,spectrometer_temp_neo,user_id,version,white_ref_data_mini,white_ref_data_neo,white_ref_scan_time,raw_data,dark_ref_data,white_ref_data,scope_data,specimen_id,lot_name,product_id,product_name
lot_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
111523,1,117,"[{""wavelength"": 287.7815856933594, ""reflectanc...","[{""wavelength"": 927.305419921875, ""reflectance...",1385,"[-3.236438512802124, -2.536426067352295, -1.06...","[42.75387191772461, 22.02066421508789, 27.4696...",1762494887,1762494990,-273.15,True,True,665219,0.95,12,111523-904635,"[12.064212620258331, 13.021287298202514, 13.06...","[930.9378662109375, 1001.3391723632812, 1216.2...",33.699677,30.878265,1957,Q,"[16.801546096801758, 15.982357025146484, 18.45...","[1183.458984375, 1276.8721923828125, 1549.7596...",1762494924,None,None,None,None,665219,SP68,6971,Soy Protein Powder
111523,1,117,"[{""wavelength"": 287.7815856933594, ""reflectanc...","[{""wavelength"": 927.305419921875, ""reflectance...",1385,"[-3.236438512802124, -2.536426067352295, -1.06...","[42.75387191772461, 22.02066421508789, 27.4696...",1762494887,1762495032,-273.15,True,True,665220,0.95,12,111523-38296,"[11.992133670300245, 11.464288158714771, 11.95...","[998.419189453125, 1073.089111328125, 1302.574...",33.699677,30.878265,1957,Q,"[16.801546096801758, 15.982357025146484, 18.45...","[1183.458984375, 1276.8721923828125, 1549.7596...",1762494924,None,None,None,None,665220,SP68,6971,Soy Protein Powder
111523,1,117,"[{""wavelength"": 287.7815856933594, ""reflectanc...","[{""wavelength"": 927.305419921875, ""reflectance...",1385,"[-3.236438512802124, -2.536426067352295, -1.06...","[42.75387191772461, 22.02066421508789, 27.4696...",1762494887,1762495073,-273.15,True,True,665225,0.95,12,111523-614002,"[9.664831161499023, 10.020500552654266, 10.387...","[940.2338256835938, 1010.7279052734375, 1227.6...",33.699677,30.878265,1957,Q,"[16.801546096801758, 15.982357025146484, 18.45...","[1183.458984375, 1276.8721923828125, 1549.7596...",1762494924,None,None,None,None,665225,SP68,6971,Soy Protein Powder
111523,1,117,"[{""wavelength"": 287.7815856933594, ""reflectanc...","[{""wavelength"": 927.305419921875, ""reflectance...",1385,"[-3.236438512802124, -2.536426067352295, -1.06...","[42.75387191772461, 22.02066421508789, 27.4696...",1762494887,1762495111,-273.15,True,True,665227,0.95,12,111523-549703,"[10.587818872928619, 11.552852725982666, 12.68...","[976.9583129882812, 1047.412841796875, 1274.99...",33.699677,30.878265,1957,Q,"[16.801546096801758, 15.982357025146484, 18.45...","[1183.458984375, 1276.8721923828125, 1549.7596...",1762494924,None,None,None,None,665227,SP68,6971,Soy Protein Powder
111523,1,117,"[{""wavelength"": 287.7815856933594, ""reflectanc...","[{""wavelength"": 927.305419921875, ""reflectance...",1385,"[-3.236438512802124, -2.536426067352295, -1.06...","[42.75387191772461, 22.02066421508789, 27.4696...",1762494887,1762495156,-273.15,True,True,665228,0.95,12,111523-50769,"[10.68888391852379, 12.044209411740303, 13.424...","[960.2284545898438, 1029.8740234375, 1253.2399...",33.699677,30.905304,1957,Q,"[16.801546096801758, 15.982357025146484, 18.45...","[1183.458984375, 1276.8721923828125, 1549.7596...",1762494924,None,None,None,None,665228,SP68,6971,Soy Protein Powder
111525,1,117,"[{""wavelength"": 287.7815856933594, ""reflectanc...","[{""wavelength"": 927.305419921875, ""reflectance...",1385,"[-3.236438512802124, -2.536426067352295, -1.06...","[42.75387191772461, 22.02066421508789, 27.4696...",1762494887,1762495798,-273.15,True,True,665244,0.95,12,111525-813626,"[11.760697156190872, 13.316925656795501, 13.25...","[1015.1068115234375, 1089.306884765625, 1320.9...",33.811493,31.094818,1957,Q,"[16.801546096801758, 15.982357025146484, 18.45...","[1183.458

(20, 33)


# by batch

In [ ]:
batch = 5028
today_date = datetime.now().strftime("%y%m%d")
print (today_date)

## label

In [ ]:
BASE_QUERY = f"""
SELECT lnpv.lot_id, lot.name as lot_name, 
	   property.name as property_name,
       lnpv.value as property_value,
       lot.company_id,
       lot.product_type_id,
       product_type.name as product_name
FROM lot_numerical_property_value lnpv
INNER JOIN lot on lot.id = lnpv.lot_id
INNER JOIN numerical_property on numerical_property.id = lnpv.numerical_property_id
INNER JOIN property on numerical_property.property_id = property.id
INNER JOIN product_type on product_type.id = lot.product_type_id
INNER JOIN lot_batch_bridge br on br.lot_id = lnpv.lot_id
WHERE br.lot_batch_id IN ({batch})

UNION ALL

SELECT lopv.lot_id, lot.name as lot_name, 
	   property.name as property_name,
       lopv.value as property_value,
       lot.company_id,
       lot.product_type_id,
       product_type.name as product_name
FROM lot_option_property_value lopv
INNER JOIN lot on lot.id = lopv.lot_id
INNER JOIN option_property on option_property.id = lopv.option_property_id
INNER JOIN property on option_property.property_id = property.id
INNER JOIN product_type on product_type.id = lot.product_type_id
INNER JOIN lot_batch_bridge br on br.lot_id = lopv.lot_id
WHERE br.lot_batch_id IN ({batch})

ORDER BY lot_id
"""

In [ ]:
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY).set_index("lot_id")
print("successful query")

info_filename = output_folder / f"info_batch{batch}_{today_date}.csv"
info_df.to_csv(info_filename)

## spectra

In [ ]:
BASE_QUERY = f"""
SELECT
	sp.id AS specimen_id,
	l.id AS lot_id,
	l.name AS lot_name,
	sp.date_scanned,
	sp.analyzer_id AS analyzer_id
FROM
	specimen sp
	INNER JOIN lot l ON l.id = sp.lot_id
    INNER JOIN lot_batch_bridge br on br.lot_id = l.id
WHERE
	# l.company_id = 1243
    br.lot_batch_id IN ({batch})
ORDER BY
	l.id;
"""

In [ ]:
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY)
print("successful query")
efs_conn = BaseEFSConnector(url=PF_EFS_URL)
print(len(info_df))

spectra_df = efs_conn.fetch_spectra(info_df.specimen_id.values)
column_diff = set(info_df.columns) & set(spectra_df.columns)
joined_df = spectra_df.merge(
    info_df.drop(column_diff, axis=1), right_on="specimen_id", left_index=True
)
joined_df.set_index("lot_id", inplace=True)

print("Successfully queried from DB")
display(joined_df.head(n=10))
print(joined_df.shape)

spectra_filename = output_folder / f"spectra_batch{batch}_{today_date}.csv"
joined_df.to_csv(spectra_filename)